# GrabCutによる一軍画像のセグメンテーションバッチ処理プログラム


**Usage**:

    df = pd.read_excel('画像リストUTF8.xlsx')
    getSilhouette(df, savedir='シルエット',smooth=False, interactive = True, test=False)
　
**パラメータ**
>    **df** バッチ処理指示用 pandas dataFrame  
>    **savedir** 保存先フォルダのパス  
>    **smooth** 領域抽出に先立って画像をぼかすかどうかのフラグ   
>            　　　　ぼかす場合は  ssize 欄の数値が適用される  
>    **interactive** 結果を１枚ずつ確認するかどうか  
>    **test** テストモードかどうかのフラグ   
            テストモードの場合は check 欄に１のある画像だけが処理対象となる  

次のようなバッチ司令ファイルを用意する

In [5]:
# バッチ司令ファイルの読み込み
df = pd.read_excel('画像リストUTF8.xlsx')
# df = pd.read_csv('画像リストUTF8.csv', sep=',')
df.head(10)

,topdir,subdir,filename,Area,offset,check,ssize,ckernel,okernel,reverse,備考
0,一軍,17Apically,17daruma1o03_l.jpg,1,20,NaN,NaN,NaN,NaN,NaN,NaN
1,一軍,17Apically,17daruma1o08_l.jpg,1,25,NaN,NaN,NaN,NaN,NaN,NaN
2,一軍,17Apically,17daruma1o09_l.jpg,1,20,NaN,NaN,NaN,NaN,NaN,NaN
3,一軍,17Apically,17daruma4o02_l.jpg,1,20,NaN,NaN,NaN,NaN,NaN,NaN
4,一軍,17Apically,17daruma4o06_l.jpg,1,20,NaN,NaN,NaN,NaN,NaN,NaN
5,一軍,17Apically,17daruma4o08_l.jpg,1,10,NaN,NaN,NaN,NaN,NaN,NaN
6,一軍,17Apically,17daruma5o03_l.jpg,1,20,NaN,25.0,33.0,NaN,1.0,NaN
7,一軍,17Apically,17daruma6o06_l.jpg,1,20,NaN,15.0,33.0,NaN,1.0,NaN
8,一軍,17Apically,17tohodm1o05_l.jpg,1,20,NaN,15.0,33.0,NaN,1.0,NaN
9,一軍,17Conv_tri,17fuyuji1o05_l.jpg,1,10,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import math
import pandas as pd

# Grabcut をマージン5%だけを背景指定して得られる画像を返す
def grabcut(img, offset=5, smooth=False, ssize=5):
    h,w = img.shape[0],img.shape[1]
    offsetY,offsetX = int(h*offset/100),int(w*offset/100)
    bgdmodel = np.zeros((1,65),np.float64)
    fgdmodel = np.zeros((1,65),np.float64)
    mask = np.zeros(img.shape[:2],np.uint8)
    rect=(offsetX,offsetY,img.shape[1]-2*offsetX,img.shape[0]-2*offsetY)
    # print(rect,img.shape,mask.shape)
    
    if(smooth): # ガウスぼかしをかけるかどうか
        img = cv2.GaussianBlur(img,(ssize,ssize),0)
    
    cv2.grabCut(img,mask,rect,bgdmodel,fgdmodel,1,cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    img = img*mask2[:,:,np.newaxis]
    
    return img

# grabcut されていることが前提で、背景は完全黒である画像の完全黒部分を余白としてカットする
def margincut(img,pad=5, needRect=False):
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = np.copy(img)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    _img,contours, _hcy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
    x1,x2,y1,y2 = [],[],[],[]
    for i in range(len(contours)):
        # ret = (x, y, w, h)
        ret = cv2.boundingRect(contours[i])
        x1.append(ret[0])
        y1.append(ret[1])
        x2.append(ret[0] + ret[2])
        y2.append(ret[1] + ret[3])
        
    x_min = min(x1)
    y_min = min(y1)
    x_max = max(x2)
    y_max = max(y2)
    
    # pad 分だけ余白をつけてコピー
    objecth = y_max-y_min
    objectw = x_max-x_min
    padY,padX = int(objecth*pad/100),int(objectw*pad/100) 
    if len(img.shape) == 3:
        newimg = np.zeros((objecth+2*padY, objectw+2*padX,3),np.uint8)
    else:
        newimg = np.zeros((objecth+2*padY, objectw+2*padX),np.uint8)
    newimg[padY:padY+objecth,padX:padX+objectw] = img[y_min:y_max,x_min:x_max].copy()    
    if needRect == True:
        return newimg,  (x_min,y_min,x_max,y_max)
    else:
        return newimg
    
# プロット用関数
def plotimg(img,layout="111"):
    if IMGON is False: return
    if len(img.shape) == 2:
        pltgry(img,layout)
    elif len(img.shape) ==3:
        pltcol(img,layout)

def pltgry(img,layout="111"):
    plt.subplot(layout)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_GRAY2RGB))

def pltcol(img,layout="111"):
    plt.subplot(layout)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    
# ２枚の画像をサイズを並べた画像を作成する
def mkparaimage(img1,img2):
    h1,w1 = img1.shape[:2]
    h2,w2 = img2.shape[:2]
    if img1.ndim == 2:
        img11 = np.zeros((h1,w1,3))
        img11[:,:,0]=img11[:,:,1]=img11[:,:,2]=img1
    else:
        img11=img1
    if img2.ndim == 2:
        img22 = np.zeros((h2,w2,3))
        img22[:,:,0]=img22[:,:,1]=img22[:,:,2]=img2
    else:
        img22=img2
    paraimg = 255*np.ones((max(h1,h2),w1+w2+10,3),dtype=np.uint8)
    
    paraimg[0:h1,0:w1,:] = img11
    paraimg[0:h2,w1+10:,:]=img22
    
    return paraimg

# mkparaimage で２枚並べた画像を表示
def imshowpara(img1,img2):
    plotimg(mkparaimage(img1,img2))
            
# 画像サイズの縦か横の大きい方が指定サイズになるようにリサイズする。
def resize(img, size=512):
    maxsize = np.max(np.array(img.shape[:2]))
    height = size*img.shape[0]//maxsize
    width = size*img.shape[1]//maxsize
    output = np.zeros((height+40, width+40,3),np.uint8)
    output[20:20+height,20:20+width]=cv2.resize(img,(width,height))
    return output

# ０画素を赤表示に変える
def redmasked(img):
    ones = np.ones(img.shape[:2],np.uint8)
    zeros = np.zeros(img.shape[:2],np.uint8)
    ret,img2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV)
    b,g,r = cv2.split(img2)
    red = cv2.bitwise_and(ones,b)
    red = cv2.bitwise_and(red,g)
    red = cv2.bitwise_and(red,r)
    red = cv2.merge((zeros,zeros,red*128))
    red = cv2.bitwise_or(red,img)
    return red

#  指定した中心周りに指定した角度だけ回転したを返す
def imagerotation(img, deg,cx= -1, cy=-1 , ratio=4):  # deg 回転角，　(cx,cy) 回転中心  ratio
    if cx < 0:
        cx = img.shape[1]/2
        cy = img.shape[0]/2
    rows,cols = img.shape[:2] 
    need = int(np.sqrt(rows**2+cols**2)) # 描画エリアを十分確保するために対角の長さを計算
    xoff = int((need-cols)/2)
    yoff = int((need-rows)/2)
    if len(img.shape)==3:
        tmpimg = np.zeros((need,need,3),dtype=np.uint8) # 描画用画像エリア
    else:
        tmpimg = np.zeros((need,need),dtype=np.uint8) # 描画用画像エリア
    tmpimg[yoff:yoff+rows,xoff:xoff+cols]=img # シルエット画像をコピー
    mat = cv2.getRotationMatrix2D((cx+xoff,cy+yoff), deg, 1.0)
    outimg = cv2.warpAffine(tmpimg, mat, (0,0))
    outimg = margincut(outimg,pad=ratio, needRect=False)
    return outimg
        
# ラベリングについては、　https://dronebiz.net/tech/opencv/labeling


In [7]:
import os

def batch(savedir='シルエット',smooth=False, interactive = True, test=False):
    # savedir 保存先
    # smooth 領域抽出に先立って画像をぼかすかどうかのフラグ ぼかす場合は excel ファイルの ssize 欄の数値が適用される
    # interactive 結果を１枚ずつ確認するかどうか
    # test テストモードかどうかのフラグ
    # テストモードの場合は check 欄に１のある画像だけが処理対象となる
    
    for i in range(len(df)):
        idata = df.iloc[i]
        topdir = idata['topdir']  #  画像ファイルのパスのベース
        subdir = idata['subdir']  #  サブディレクトリ
        filename = idata['filename'] #  ファイル名
        arean = idata['Area'] # 何番目に大きいエリアを抽出するか。off セットに左右される
        off = idata['offset'] # 　グラブカットする時に確実に背景であると指定する余白の割合（％　< 50）。大根が入るギリギリが望ましいが適当で良い。
        cksize = idata['ckernel'] # クロージング（穴埋）回数　１ピクセルなら１度で埋まる
        oksize = idata['okernel'] # オープニング（ヒゲ除去）回数　１ピクセルなら１度で除去される
        reverse = idata['reverse'] # 通常は　オープニング　→ クロージングの順だが、逆順にするかどうか
                
        check = idata['check'] #  処理対象かどうかのフラグ　　test がTrueの時のみ意味がある
        ssize = idata['ssize'] # GrabCut 処理の前に平滑化を施す平滑化のカーネルサイズ　smooth が True の場合のみ意味がある
        
        # ssize,cksize,oksize,reverse はデフォルト値を適用  # 指定していないと NaNとなるが、NaNは float 扱いであることを利用
        ssize =  5 if pd.isnull(ssize)  else int(ssize)
        cksize = 3 if pd.isnull(cksize)  else int(cksize) 
        oksize = 3  if pd.isnull(oksize)  else int(oksize)
        reverse = 0 if pd.isnull(reverse)  else int(reverse)
        
        ckernel = np.ones((cksize,cksize),np.uint8) 
        okernel = np.ones((oksize,oksize),np.uint8)

        if test and check !=1 : #  test 時で check が 1 でない画像はスルーする
            continue 
                         
        path = os.path.join(topdir,subdir,filename)
        print(path)
        
        # カラー画像としてソース画像を読み込み、グラブカットで初期切り出しを行う。
        src = cv2.imread(path, cv2.IMREAD_COLOR)
        cimg = grabcut(src, offset=off, smooth=smooth, ssize=ssize)
        
        # 抽出されたエリアを２値化する
        cimg2 = cv2.cvtColor(cimg,cv2.COLOR_BGR2GRAY)
        ret, cntimg = cv2.threshold(cimg2, 0, 255, cv2.THRESH_BINARY|cv2.THRESH_OTSU) # 真っ黒以外は白扱い

        # 白領域にラベルづけし、領域ごとの特徴量を調べる
        # 先立って、オープニングで外部ノイズを除去、クロージングで内部の穴や溝を埋める。
        
        if reverse != 1: # 表面に色がついている場合、孔があきやすいのでクロージングを先にするように指定した方が良い
            cntimg = cv2.morphologyEx(cntimg, cv2.MORPH_OPEN, kernel = okernel)
            cntimg = cv2.morphologyEx(cntimg, cv2.MORPH_CLOSE, kernel = ckernel)
        else:
            cntimg = cv2.morphologyEx(cntimg, cv2.MORPH_CLOSE, kernel = ckernel)
            cntimg = cv2.morphologyEx(cntimg, cv2.MORPH_OPEN, kernel = okernel)
            
        nLabels, labelImages, data, center = cv2.connectedComponentsWithStats(cntimg) # ラベルづけ
        sortedindex = np.argsort(data[:,4])[::-1] # 面積の大きい順に領域番号を並べたリスト
        x1,y1,w,h = data[sortedindex[arean]][:4] # 面積がCSVファイル記載の順位である白領域のバウンダリBOXをえる
        # ２ピクセルだけ多めに
        x1 = x1-2 if x1>2 else x1
        y1 = y1-2 if y1>2 else y1
        w = w+4 if x1+w+4<src.shape[1] else src.shape[1]
        h = h+4 if y1+h+4<src.shape[0] else src.shape[0]
        targetRectImage = cntimg[y1:y1+h,x1:x1+w] # その領域のみを切り出した画像を作成
        
        # 以下は内部にある穴埋めのための処理
        # 根を切り出した画像の輪郭線を抽出
        _img,contours,hierarchy = cv2.findContours(targetRectImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        contours.sort(key=cv2.contourArea, reverse=True) # 面積の大きい順に並べ替え
        dimg = np.zeros_like(targetRectImage)
        
        # 一番大きい領域を描いて塗りつぶす。
        dimg = cv2.drawContours(dimg,contours[0:1],0,(255,255,255),thickness=-1)
        os.makedirs(os.path.join(savedir,subdir), exist_ok=True) # 保存先フォルダがなければ作成
        cv2.imwrite(os.path.join(savedir,subdir,filename),dimg)
        # print(os.path.join(savedir,os.path.basename(path)))
        
        if interactive:
            ppimg = src.copy()
            cv2.rectangle(ppimg,(x1,y1),(x1+w,y1+h),(255,0,0),2)
            cv2.imshow(filename,mkparaimage(ppimg,dimg))
            key = cv2.waitKey(0)
            cv2.destroyAllWindows()
            cv2.waitKey(1) 
            if key == 113: #  "Q" で終了する
                break

In [8]:
df = pd.read_excel('画像リストUTF8.xlsx')
batch(smooth=True,interactive=True,test=True) 

一軍/17Elliptic/17okinaw1o05_l.jpg


In [11]:
dir,files = os.path.split("b/c/c")
_,subdir = os.path.split(dir)
os.path.join("A",subdir,files)

'A/c/c'